In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import utm

vancouver_path = "./vancouvercrime.csv"
crime_mapping_path = "./crime_type_mapping.csv"
eventDataVancouver = pd.read_csv('Vancouver_events.csv')
df = pd.read_csv(vancouver_path)[:50000]
df = df[df['YEAR'].isin([2015,2016,2017,2018,2019])]
crime_code_df = pd.read_csv(crime_mapping_path)

In [ ]:
crime_list = crime_code_df["Vancouver"].tolist()
# Source https://vancouver.ca/police/Planning/2019/15-19CR.pdf
crime_rate_list ={"2015":7.358,"2016":7.895,"2017":7.786,"2018":7.835,"2019":8.282}

In [ ]:
loc_df_list=[]
crime_df_list=[]
fact_df_list=[]
#Event Dimension
event_df = pd.DataFrame(columns=['event_key','event_name','event_type','event_location','event_neighbourhood','event_city','event_start_date','event_end_date','event_location_size'])
for idx_event, row_event in eventDataVancouver.iterrows():
    event_row=[idx_event+30000000]
    event_row += [row_event['NAME'],row_event['TYPE'],row_event['LOCATION'],row_event['NEIGHBOURHOOD'],'Denver',row_event['DATE_START'],row_event['DATE_END'],row_event['LOCATION-SIZE']]
    event_df.loc[len(event_df)] = event_row

In [ ]:
event_start_lst = event_df["event_start_date"].tolist()
event_end_lst = event_df["event_end_date"].tolist()

In [ ]:
for i, row in df.iterrows():
    # index = len(fact_df)
    index = len(loc_df_list)
    na = np.nan
    
    # Van id starts from 10^7, Den id starts from 10^6
    location_id = index + 10000000
    city = "Vancouver"
    location = row["HUNDRED_BLOCK"]
    neighborhood = row["NEIGHBOURHOOD"]
    lat,lon = utm.to_latlon(row['X'],row['Y'],10,'U') if row['X']>0 and row['Y']>0 else (na,na)
    
    
    
    crime_id = index + 20000000
    
    year = row['YEAR']
    month = row['MONTH']
    day = row['DAY']
    hour = row['HOUR']
    minute = row['MINUTE']

    crime_type_map = crime_code_df.iloc[crime_list.index(row['TYPE'])]
    crime_category_name = crime_type_map["corr Denver category"]
    crime_type_name = crime_type_map["corr Denver type"]
    crime_detail = crime_type_map["corr Denver detail"]
    crime_rate = crime_rate_list[str(year)]
    
    str_month = str(month) if month>9 else '0'+str(month)
    str_day = str(day) if day>9 else '0'+str(day)
    date_id = int(str(year)+str_month+str_day)
    
    
    #Event
    event_id = np.nan
    #for idx_event, row_event in event_df.iterrows():
    for i in range(len(event_start_lst)):
        #the crime is in the event period
        if date_id>=event_start_lst[i] and date_id<=event_end_lst[i]:
            row_event = event_df.iloc[i]
            local_location = location.upper() if type(location)==str else "(｡･ω･)ﾉﾞ"
            event_location = row_event['event_location'].upper() if type(row_event['event_location'])==str else "ヾ(´･ω･｀)ﾉ"
            if local_location in event_location:
                event_id = row_event['event_key']
                break
            elif neighborhood == row_event['event_neighbourhood']:
                event_id = row_event['event_key']
            elif row_event['event_location_size'] == 'city' and np.isnan(event_id):
                event_id = row_event['event_key']
    
    is_night = 0 if 7 <= hour <= 18 else 1
    
    fatal_keywords=['FATAL','DEAD','DEATH','KILL','HOMICIDE','MURDER']
    is_fatal = 0
    for keyword in fatal_keywords:
        for name in [crime_category_name,crime_type_name,crime_detail]:
            if keyword in name.upper():
                is_fatal = 1

    # location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    loc_df_list.append([location_id,city,location,neighborhood,lat,lon,crime_rate])
    #crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,hour,minute,0,na,na,na,na,na,na]
    crime_df_list.append([crime_id,crime_category_name,crime_type_name,crime_detail,hour,minute,0,na,na,na,na,na,na])
    #fact_df.loc[index] = [location_id,crime_id,date_id,event_holiday_id,crime_type_map["is_traffic"],crime_type_map['is_fatal'],is_night]
    fact_df_list.append([location_id,crime_id,date_id,event_id,crime_type_map["is_traffic"],crime_type_map['is_fatal'],is_night])


In [ ]:
fact_df = pd.DataFrame(fact_df_list, columns=['location_Key','crime_key', 'date_key','event_key', 'is_traffic','is_fatal','is_nighttime'],dtype="Int64")
location_df = pd.DataFrame(loc_df_list, columns=['location_Key','city','location_name', 'neighborhood', 'latitude','longitude','crime_rate_percentage'])
crime_df= pd.DataFrame(crime_df_list, columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])

In [ ]:
crime_df.head()

In [ ]:
location_df.head()

In [ ]:
fact_df.head()